# Zjawisko przeuczenia + regularyzacja
## [Zadanie](http://pages.mini.pw.edu.pl/~karwowskij/mioad/lab-sieci.html#org6058800)

Zaimplementować mechanizm regularyzacji wag w sieci oraz mechanizm zatrzymywania uczenia przy wzroście błędu na zbiorze walidacyjnym.

Przeprowadzić eksperymenty na zbiorach i porównać skuteczność na zbiorze testowym dla różnych wariantów przeciwdziałania przeuczeniu sieci:
- multimodal-sparse,
- rings5-sparse,
- rings3-balance,
- xor3-balance.

In [1]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
#### Miscellaneous classes

class Sigmoid:
    @staticmethod
    def activate(z):
        return 1 / (1 + np.exp(-z))

    @staticmethod
    def derivative(z):
        return Sigmoid.activate(z) * (1 - Sigmoid.activate(z))

class Linear:
    @staticmethod
    def activate(z):
        return z

    @staticmethod
    def derivative(z):
        return np.ones(shape = z.shape)
    
class MSECostFunction:
    @staticmethod
    def cost(y_true, y_pred):
        return np.mean((y_pred - y_true) ** 2)

    @staticmethod
    def derivative(y_true, y_pred):
        return y_pred - y_true
    
class SoftMax:
    @staticmethod
    def activate(z):
        e_x = np.exp(z - np.max(z))
        return e_x / e_x.sum(axis=0, keepdims=True)
    
    @staticmethod
    def derivative(z):
        return 1

class Tanh:
    @staticmethod
    def activate(z):
        return np.tanh(z)
    
    @staticmethod
    def derivative(z):
        return 1 - np.power(np.tanh(z), 2)
    
class ReLU:
    @staticmethod
    def activate(z):
        return np.maximum(0, z)
    
    # Not everyone implements the exact gradient for the ReLU function
    # due to the non-differentiability of this function at 0 
    @staticmethod
    def derivative(z):
        dr = np.array(z)
        dr[dr>0] = 1
        dr[dr<0] = 0
        # non-differentiability at 0
        eps = 1e-5
        dr[dr==0] = eps
        return dr

In [3]:
class MLP:

    def __init__(self, layers, initial_dist, activation_functions, softmax, cost_function=MSECostFunction):
        """
        Presence of at least one hidden layer is an accompanying assumption        

        Takes:        
        layers - list of numbers of neurons in subsequent layers
        activation_functions - list of names of functions ie ('sigmoid', 'linear')

        Remarks: 
            - Length of layers list should be equal to length of activation_functions list + 1 
            - The biases and weights for the network are initialized randomly, using continuous uniform 
              distribution with certain bounds between 0 and 1 or -1 and 1 or Gaussian distribution with mean 0, 
              and variance 1.
        """
        
        self.layers = layers
        self.n_layers = len(layers)
        self.weights = []
        self.biases = []
        self.weights_bias_init(initial_dist)

        self.activation_functions = []
        self.activation_functions_derivatives_per_layer = []
        for fun in activation_functions:
            self.activation_functions.append(fun.activate)
            self.activation_functions_derivatives_per_layer.append(fun.derivative)

        self.cost_function = cost_function
        self.softmax = softmax

        if softmax:
            self.activation_functions[-1] = activation_functions[-1].activate
            
    def weights_bias_init(self, initial_dist):
        if initial_dist == "gaussian":
            self.biases = [np.random.normal(size=(y, 1)) for y in self.layers[1:]]
            self.weights = [np.random.normal(size=(y, x)) for x, y in zip(self.layers[:-1], self.layers[1:])]
            
        elif initial_dist == "uniform":
            self.biases = [np.random.uniform(-1, 1, size=(y, 1)) for y in self.layers[1:]]
            self.weights = [np.random.uniform(-1, 1, size=(y, x)) for x, y in zip(self.layers[:-1], self.layers[1:])]
            
        # Due to poor network performance with gaussian and uniform distributions in classification 
        # problems, I decided to implement xavier initialization 
        elif initial_dist == "xavier":
            self.biases = [np.random.uniform(-math.sqrt(6)/ math.sqrt(self.layers[i] + self.layers[i + 1]),
                    math.sqrt(6)/ math.sqrt(self.layers[i] + self.layers[i + 1]),size=(self.layers[i + 1], 1))
                for i in range(self.n_layers - 1)]
            self.weights = [np.random.uniform(-math.sqrt(6)/ math.sqrt(self.layers[i] + self.layers[i + 1]),
                    math.sqrt(6)/ math.sqrt(self.layers[i] + self.layers[i + 1]), size=(y, x))
                for x, y, i in zip(self.layers[:-1], self.layers[1:], range(self.n_layers))]
        else:
            self.biases = [np.random.uniform(0, 1, size=(y, 1)) for y in self.layers[1:]]
            self.weights = [np.random.uniform(0, 1, size=(y, x)) for x, y in zip(self.layers[:-1], self.layers[1:])]


    def forward(self, x):
        """
        Returns the output of the network if x is an input.
        """
        flat_values = []
        activation_values = [np.copy(x)]
        for w, b, f in zip(self.weights, self.biases, self.activation_functions):
            flat_values.append(w.dot(activation_values[-1]) + b)
            activation_values.append(f(flat_values[-1]))
        return (flat_values, activation_values)
            
            
    def backprop(self, x_batch, y_batch):
        """
        Function performing backpropagation
        Returns nabla_b and nabla_w representing the
        derivatives by weights and biases respectively. 
        nabla_b and nabla_w are calculated layer-by-layer.
        """ 
        (flat_values, activation_values) = self.forward(x_batch)
        errors = [None] * len(self.weights)
        
        
        if not self.softmax:
            errors[-1] = (-self.cost_function.derivative(y_batch, activation_values[-1])) * self.activation_functions_derivatives_per_layer[-1](flat_values[-1])
        else:
            errors[-1] = y_batch - activation_values[-1]
            
        for i in reversed(range(len(errors) - 1)):
            errors[i] = self.weights[i + 1].T.dot(errors[i + 1]) * self.activation_functions_derivatives_per_layer[i](flat_values[i])
        batch_size = y_batch.shape[1]
        d_b = [e.dot(np.ones((batch_size, 1))) / float(batch_size) for e in errors]
        d_w = [e.dot(activation_values[i].T) / float(batch_size) for i, e in enumerate(errors)]
        return (d_w, d_b) 
    
    def predict(self, x):
        return self.forward(x.T)[1][-1].T
    
    # The following implementation is based on the derivation of the 
    # derivative formula for a new cost function with L2 regularization
    def train_RMSProp(self, x, y, beta, batches_num = 10, epochs = 100, eta = 0.001, regression=True, regularization = False, lamb=0.03):
       
        # training and validation sets split
        x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)
    
        if regression:
            prediction = self.predict(x_val)
            errors = [metrics.mean_absolute_error(y_val, prediction)]
        else:
            prediction = self.predict(x_val)
            prediction = np.argmax(prediction, axis=1)
            errors = [accuracy_score(np.argmax(y_val, axis = 1), prediction)]
            
        for e in range(epochs):
            grad_b = [np.zeros(b.shape) for b in self.biases]
            grad_w = [np.zeros(w.shape) for w in self.weights]

            shuffled_indexes = np.random.permutation(x_train.shape[0])
            updated_x = x_train.copy()[shuffled_indexes]
            updated_y = y_train.copy()[shuffled_indexes]

            updated_x = np.array_split(updated_x, batches_num)
            updated_y = np.array_split(updated_y, batches_num)

            for batch in range(batches_num):
                x_batch = updated_x[batch].T
                y_batch = updated_y[batch].T
                d_w, d_b = self.backprop(x_batch, y_batch)

                grad_b = [(1 - beta) * np.square(db) + beta * gb for gb, db in zip(grad_b, d_b)]
                grad_w = [(1 - beta) * np.square(dw) + beta * gw for gw, dw in zip(grad_w, d_w)]

                self.biases = [b + np.divide(db, np.sqrt(gb) + 1e-8) * eta for b, gb, db in zip(self.biases, grad_b, d_b)]
                if regularization:
                    # modification of the method of weight change
                    self.weights = [w*(1-eta*lamb/y_batch.shape[1]) + np.divide(dw, np.sqrt(gw) + 10e-8) * eta for w, gw, dw in zip(self.weights, grad_w, d_w)]
                else: 
                    self.weights = [w + np.divide(dw, np.sqrt(gw) + 1e-8) * eta for w, gw, dw in zip(self.weights, grad_w, d_w)]
            if regression:
                prediction = self.predict(x_val)
                errors.append(metrics.mean_absolute_error(y_val, prediction))
            else:
                prediction = np.argmax(self.predict(x_val), axis = 1)
                errors.append(accuracy_score(np.argmax(y_val, axis = 1), prediction))
            # I assume the number of epochs is at least 50
            # Regularization protects us from too high weight values,
            # as in each iteration they are additionally lowered by their percentage
            if e > 50:
                # Stop at a 10% difference between the current error value and the min/max error value 
                # All (ie min, max, current) errors are calculated on validation set for each epoch
                if regression:
                    if errors[e]>1.1*max(errors):
                        return 
                else:
                    if errors[e]<0.9*min(errors):
                        return 

## 1. Regression

### 1.1. Multimodal-sparse dataset

In [4]:
multimodal_large_train = pd.read_csv("../data/mio1/regression/multimodal-sparse-training.csv")[["x", "y"]]
multimodal_large_test = pd.read_csv("../data/mio1/regression/multimodal-sparse-test.csv")[["x", "y"]]

x_test = multimodal_large_test["x"].to_numpy().reshape(-1, 1)
y_test = multimodal_large_test["y"].to_numpy().reshape(-1, 1)

y_train = multimodal_large_train["y"].to_numpy().reshape(-1, 1)
x_train = multimodal_large_train["x"].to_numpy().reshape(-1, 1)

#### 1.1.1. With regularization

In [5]:
scores = []

for i in range(50):
    mlp = MLP(layers=[1,40,1], initial_dist="default", activation_functions=[Sigmoid, Linear], softmax=False)
    weights = mlp.weights
    biases = mlp.biases

    mlp.train_RMSProp(x=x_train, y=y_train, eta=0.01, epochs=2000, batches_num=5, regularization=True, beta=0.9)
    prediction = mlp.predict(x_test)
    mae_score = metrics.mean_absolute_error(prediction, y_test)
    scores.append(mae_score)

In [6]:
pd.DataFrame({'Mean MAE score': np.mean(scores),
              'Max MAE score': np.max(scores),
             'MAE score std dev': np.std(scores)},
            index=[0])

,Mean MAE score,Max MAE score,MAE score std dev
0,7.255764,7.868503,0.1852


#### 1.1.2. Without regularization

In [7]:
scores = []

for i in range(50):
    mlp = MLP(layers=[1,40,1], initial_dist="default", activation_functions=[Sigmoid, Linear], softmax=False)
    weights = mlp.weights
    biases = mlp.biases

    mlp.train_RMSProp(x=x_train, y=y_train, eta=0.01, epochs=2000, batches_num=5, regularization=False, beta=0.9)
    prediction = mlp.predict(x_test)
    mae_score = metrics.mean_absolute_error(prediction, y_test)
    scores.append(mae_score)

In [8]:
pd.DataFrame({'Mean MAE score': np.mean(scores),
              'Max MAE score': np.max(scores),
             'MAE score std dev': np.std(scores)},
            index=[0])

,Mean MAE score,Max MAE score,MAE score std dev
0,8.016637,8.955756,0.253122


## 2. Classification

### 2.1. Rings3 dataset

In [9]:
rings3_train = pd.read_csv("../data/mio1/classification/rings3-balance-training.csv")
rings3_test = pd.read_csv("../data/mio1/classification/rings3-balance-test.csv")

x_test = rings3_test[["x", "y"]].to_numpy()
x_train = rings3_train[["x", "y"]].to_numpy()

y_test = rings3_test["c"] * 1
n = np.max(y_test) + 1
y_test = np.eye(n)[y_test.T]

y_train = rings3_train["c"] * 1
n = np.max(y_train) + 1
y_train = np.eye(n)[y_train.T]

#### 2.1.1. With regularization

In [10]:
scores = []

for i in range(50):
    mlp = MLP([2,20,20,3], "xavier", [Sigmoid, Sigmoid, SoftMax], True)
    weights = mlp.weights
    biases = mlp.biases

    mlp.train_RMSProp(x = x_train, y = y_train, eta = 0.001, epochs = 1000, batches_num= 10, beta=0.9, regression=False, regularization=True)
    prediction = mlp.predict(x_test)
    prediction = np.argmax(prediction, axis = 1)
    acc_score = metrics.accuracy_score(np.argmax(y_test, axis = 1), prediction)
    scores.append(acc_score)

In [11]:
pd.DataFrame({'Mean Accuracy score': np.mean(scores),
              'Max Accuracy score': np.max(scores),
             'Accuracy score std dev': np.std(scores)},
            index=[0])

,Mean Accuracy score,Max Accuracy score,Accuracy score std dev
0,0.72509,0.7655,0.014742


#### 2.1.2. Without regularization

In [12]:
scores = []

for i in range(50):
    mlp = MLP([2,20,20,3], "xavier", [Sigmoid, Sigmoid, SoftMax], True)
    weights = mlp.weights
    biases = mlp.biases

    mlp.train_RMSProp(x = x_train, y = y_train, eta = 0.001, epochs = 1000, batches_num= 10, beta=0.9, regression=False, regularization=False)
    prediction = mlp.predict(x_test)
    prediction = np.argmax(prediction, axis = 1)
    acc_score = metrics.accuracy_score(np.argmax(y_test, axis = 1), prediction)
    scores.append(acc_score)

In [13]:
pd.DataFrame({'Mean Accuracy score': np.mean(scores),
              'Max Accuracy score': np.max(scores),
              'Accuracy score std dev': np.std(scores)},
            index=[0])

,Mean Accuracy score,Max Accuracy score,Accuracy score std dev
0,0.71712,0.7455,0.012418


### 2.2. Rings5 dataset

In [14]:
rings5_train = pd.read_csv("../data/mio1/classification/rings5-sparse-training.csv")
rings5_test = pd.read_csv("../data/mio1/classification/rings5-sparse-test.csv")

x_test = rings5_test[["x", "y"]].to_numpy()
x_train = rings5_train[["x", "y"]].to_numpy()

y_test = rings5_test["c"] * 1
n = np.max(y_test) + 1
y_test = np.eye(n)[y_test.T]

y_train = rings5_train["c"] * 1
n = np.max(y_train) + 1
y_train = np.eye(n)[y_train.T]

#### 2.2.1. With regularization

In [15]:
scores = []

for i in range(50):
    mlp = MLP([2,20,20,5], "xavier", [Sigmoid, Sigmoid, SoftMax], True)
    weights = mlp.weights
    biases = mlp.biases

    mlp.train_RMSProp(x = x_train, y = y_train, eta = 0.001, epochs = 1500, batches_num = 10, beta=0.9, regression=False, regularization=True)
    prediction = mlp.predict(x_test)
    prediction = np.argmax(prediction, axis = 1)
    acc_score = metrics.accuracy_score(np.argmax(y_test, axis = 1), prediction)
    scores.append(acc_score)

In [16]:
pd.DataFrame({'Mean Accuracy score': np.mean(scores),
              'Max Accuracy score': np.max(scores),
             'Accuracy score std dev': np.std(scores)},
            index=[0])

,Mean Accuracy score,Max Accuracy score,Accuracy score std dev
0,0.75934,0.819,0.028582


#### 2.2.2. Without regularization

In [17]:
scores = []

for i in range(50):
    mlp = MLP([2,20,20,5], "xavier", [Sigmoid, Sigmoid, SoftMax], True)
    weights = mlp.weights
    biases = mlp.biases

    mlp.train_RMSProp(x = x_train, y = y_train, eta = 0.001, epochs = 1500, batches_num = 10, beta=0.9, regression=False, regularization=False)
    prediction = mlp.predict(x_test)
    prediction = np.argmax(prediction, axis = 1)
    acc_score = metrics.accuracy_score(np.argmax(y_test, axis = 1), prediction)
    scores.append(acc_score)

In [18]:
pd.DataFrame({'Mean Accuracy score': np.mean(scores),
              'Max Accuracy score': np.max(scores),
             'Accuracy score std dev': np.std(scores)},
            index=[0])

,Mean Accuracy score,Max Accuracy score,Accuracy score std dev
0,0.75169,0.7995,0.024809


### 2.3. XOR3 dataset

In [19]:
xor_train = pd.read_csv("../data/mio1/classification/xor3-balance-training.csv")
xor_test = pd.read_csv("../data/mio1/classification/xor3-balance-test.csv")

x_test = xor_test[["x", "y"]].to_numpy()
x_train = xor_train[["x", "y"]].to_numpy()

y_test = xor_test["c"] * 1
n = np.max(y_test) + 1
y_test = np.eye(n)[y_test.T]

y_train = xor_train["c"] * 1
n = np.max(y_train) + 1
y_train = np.eye(n)[y_train.T]

#### 2.3.1. With regularization

In [20]:
scores = []

for i in range(50):
    mlp = MLP([2,20,2], "xavier", [Sigmoid, SoftMax], True)
    weights = mlp.weights
    biases = mlp.biases

    mlp.train_RMSProp(x = x_train, y = y_train, eta = 0.001, epochs = 1500, batches_num = 10, beta=0.9, regression=False, regularization=True)
    prediction = mlp.predict(x_test)
    prediction = np.argmax(prediction, axis = 1)
    acc_score = metrics.accuracy_score(np.argmax(y_test, axis = 1), prediction)
    scores.append(acc_score)

In [21]:
pd.DataFrame({'Mean Accuracy score': np.mean(scores),
              'Max Accuracy score': np.max(scores),
             'Accuracy score std dev': np.std(scores)},
            index=[0])

,Mean Accuracy score,Max Accuracy score,Accuracy score std dev
0,0.723525,0.77125,0.016355


#### 2.3.2. Without regularization

In [22]:
scores = []

for i in range(50):
    mlp = MLP([2,20,2], "xavier", [Sigmoid, SoftMax], True)
    weights = mlp.weights
    biases = mlp.biases

    mlp.train_RMSProp(x = x_train, y = y_train, eta = 0.001, epochs = 1500, batches_num = 10, beta=0.9, regression=False, regularization=False)
    prediction = mlp.predict(x_test)
    prediction = np.argmax(prediction, axis = 1)
    acc_score = metrics.accuracy_score(np.argmax(y_test, axis = 1), prediction)
    scores.append(acc_score)

In [23]:
pd.DataFrame({'Mean Accuracy score': np.mean(scores),
              'Max Accuracy score': np.max(scores),
             'Accuracy score std dev': np.std(scores)},
            index=[0])

,Mean Accuracy score,Max Accuracy score,Accuracy score std dev
0,0.725775,0.76,0.018085
